## 📋 事前准备

请在您的Google Drive的「MyDrive」中创建以下文件夹结构，
并根据需要放入您想在Stable Diffusion WebUI (Forge) 中使用的数据：

```
MyDrive/
└── StableDiffusion/
    ├── checkpoints/   # 放置AI模型文件
    ├── controlnet/    # 放置ControlNet模型
    ├── embeddings/    # 放置EasyNegative等嵌入文件
    ├── lora/          # 放置LoRA模型文件
    ├── vae/           # 放置VAE模型文件
    ├── outputs/       # 生成图片的保存位置（只需创建文件夹）
    └── extensions/    # 扩展插件保存位置（会自动创建）
```

💡 **提示**：如果您想从A1111迁移样式设置，请将 `styles.csv` 文件保存到 `MyDrive/StableDiffusion/` 文件夹中。

## 🚀 使用方法

### 步骤说明：

1. **设置GPU**
   - 点击菜单栏的「运行时」→「更改运行时类型」
   - 选择GPU类型（T4、V100、A100等）

2. **执行代码**
   - 按顺序点击每个代码单元格的运行按钮（▶）
   - 等待左侧出现绿色勾号（✓）后再运行下一个
   - 或者直接选择「运行时」→「全部运行」

3. **访问WebUI**
   - 在【步骤3】执行完成后，会显示类似 `https://********.gradio.live` 的链接
   - 点击该链接即可打开Stable Diffusion WebUI

4. **停止WebUI**
   - 点击【步骤3】的停止按钮（◼）来停止WebUI

5. **重新启动**
   - 再次运行【步骤3】即可重新启动WebUI

---

## 【步骤1】连接Google Drive

将您的Google Drive挂载到Colab环境，以便访问存储在Drive中的模型文件。

**操作说明**：
1. 点击下方的运行按钮（▶）
2. 在弹出的窗口中选择「连接到Google Drive」
3. 选择您的Google账号并授权访问

In [ ]:
# 导入Google Colab的Drive模块
from google.colab import drive

# 挂载Google Drive到/content/drive路径
drive.mount('/content/drive')

print("✅ Google Drive连接成功！")

## 【步骤2】安装Stable Diffusion WebUI Forge和扩展

这一步将：
1. 下载并安装Stable Diffusion WebUI Forge
2. 让您选择要安装的扩展插件
3. 自动处理扩展的安装和更新

In [ ]:
import os
import subprocess

# 1. 克隆Stable Diffusion WebUI Forge
print("📥 正在下载Stable Diffusion WebUI Forge...")
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge.git
%cd /content/stable-diffusion-webui-forge
print("✅ WebUI Forge下载完成！\n")

# 2. 设置扩展文件夹路径
drive_extensions_path = "/content/drive/MyDrive/StableDiffusion/extensions"

# 创建extensions文件夹（如果不存在）
if not os.path.exists(drive_extensions_path):
    os.makedirs(drive_extensions_path)
    print("📁 已创建extensions文件夹")

# 3. 创建符号链接，将Google Drive的extensions文件夹链接到WebUI
!rm -rf /content/stable-diffusion-webui-forge/extensions
!ln -s /content/drive/MyDrive/StableDiffusion/extensions /content/stable-diffusion-webui-forge/extensions
print("🔗 已链接Google Drive的extensions文件夹\n")

# 4. 扩展安装界面
print("="*60)
print("📦 扩展安装器")
print("="*60)
print("请输入要安装的扩展URL（每行一个），输入完成后按Enter两次：")
print("示例: https://github.com/Bing-su/adetailer.git")
print("\n如果不输入任何内容并直接按Enter两次，将询问是否使用默认扩展列表")
print("="*60)

# 5. 收集用户输入的扩展URL
extensions = []
while True:
    line = input()
    if line:
        extensions.append(line.strip())
    else:
        break

# 6. 如果用户没有输入，询问是否使用默认列表
if not extensions:
    use_default = input("\n未输入任何扩展。是否使用默认扩展列表？(y/n): ")
    if use_default.lower() == 'y':
        extensions = [
            "https://github.com/Bing-su/adetailer.git",                            # After Detailer - 自动检测和修复人脸、手部
            "https://github.com/thomasasfk/sd-webui-aspect-ratio-helper.git",      # 宽高比助手 - 快速设置图片比例
            "https://github.com/Physton/sd-webui-prompt-all-in-one.git",          # 提示词全能工具 - 翻译、管理提示词
            "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git", # 终极放大 - 智能放大图片
            "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git",      # 标签自动补全 - 输入提示词时自动提示
            "https://github.com/KohakuBlueleaf/z-tipo-extension.git",             # Z-Tipo - 提示词增强工具
            "https://github.com/lobehub/sd-webui-lobe-theme.git",                 # Lobe主题 - 美化界面
            "https://github.com/Gourieff/sd-webui-reactor-sfw.git",               # ReActor - 人脸替换（安全版）
            "https://github.com/zixaphir/Stable-Diffusion-Webui-Civitai-Helper.git", # Civitai助手 - 模型下载和管理
        ]
        print(f"\n将安装 {len(extensions)} 个默认扩展")

# 7. 询问是否更新已存在的扩展
if extensions:
    update_existing = input("\n是否更新已存在的扩展？(y/n，默认y): ")
    update_existing = update_existing.lower() != 'n'
    
    # 8. 开始处理扩展
    print(f"\n🚀 准备处理 {len(extensions)} 个扩展...")
    print("="*60)
    
    # 切换到扩展目录
    %cd {drive_extensions_path}
    
    # 统计变量
    installed = 0
    updated = 0
    skipped = 0
    failed = 0
    
    # 9. 逐个处理扩展
    for i, ext in enumerate(extensions, 1):
        # 从URL提取扩展名称
        ext_name = ext.split('/')[-1].replace('.git', '')
        print(f"\n[{i}/{len(extensions)}] 处理: {ext_name}")
        
        # 检查扩展是否已存在
        if os.path.exists(ext_name):
            if update_existing:
                print(f"  📍 扩展已存在，正在检查更新...")
                %cd {ext_name}
                
                # 保存本地修改（如果有）
                !git stash > /dev/null 2>&1
                
                # 拉取最新代码
                result = !git pull
                
                if any('Already up to date' in line for line in result):
                    print(f"  ✅ 已是最新版本")
                    skipped += 1
                else:
                    print(f"  ✅ 更新成功")
                    updated += 1
                
                %cd ..
            else:
                print(f"  ⏭️  已存在，跳过")
                skipped += 1
        else:
            # 安装新扩展
            print(f"  📥 正在安装...")
            result = !git clone {ext} 2>&1
            
            if os.path.exists(ext_name):
                print(f"  ✅ 安装成功")
                installed += 1
            else:
                print(f"  ❌ 安装失败")
                failed += 1
    
    # 10. 返回WebUI目录
    %cd /content/stable-diffusion-webui-forge
    
    # 11. 显示处理结果统计
    print("\n" + "="*60)
    print("📊 处理完成！统计结果：")
    print(f"  ✅ 新安装: {installed} 个")
    print(f"  🔄 已更新: {updated} 个")
    print(f"  ⏭️  已跳过: {skipped} 个")
    if failed > 0:
        print(f"  ❌ 失败: {failed} 个")
    print("="*60)
    
    # 12. 显示当前已安装的所有扩展
    installed_exts = os.listdir(drive_extensions_path)
    print(f"\n📦 当前已安装的扩展 (共{len(installed_exts)}个):")
    for ext in sorted(installed_exts):
        print(f"  • {ext}")
else:
    print("\n未选择安装任何扩展。")

print("\n✨ 扩展配置完成！")

## 【步骤3】启动Stable Diffusion WebUI

**功能说明**：
- 生成的图片会自动保存到 `/content/drive/MyDrive/StableDiffusion/outputs`
- 模型文件从Google Drive读取，避免重复下载
- 支持自定义启动参数

**操作说明**：
- 点击运行按钮（▶）启动WebUI
- 点击停止按钮（◼）停止WebUI
- 再次点击运行按钮（▶）重新启动

In [ ]:
import os

# 1. 创建输出目录设置脚本
# 这个脚本会在WebUI启动时自动运行，设置图片保存路径到Google Drive
script_content = """
import os
from modules import shared

# 设置各种输出的保存路径到Google Drive
shared.opts.data['outdir_txt2img_samples'] = '/content/drive/MyDrive/StableDiffusion/outputs/txt2img-images'
shared.opts.data['outdir_txt2img_grids'] = '/content/drive/MyDrive/StableDiffusion/outputs/txt2img-grids'
shared.opts.data['outdir_img2img_samples'] = '/content/drive/MyDrive/StableDiffusion/outputs/img2img-images'
shared.opts.data['outdir_img2img_grids'] = '/content/drive/MyDrive/StableDiffusion/outputs/img2img-grids'
shared.opts.data['outdir_extras_images'] = '/content/drive/MyDrive/StableDiffusion/outputs/extras-images'

print("✅ 输出目录已设置到Google Drive")
"""

# 2. 保存脚本到WebUI的scripts文件夹
script_path = "/content/stable-diffusion-webui-forge/scripts/set_output_dir.py"
with open(script_path, "w") as f:
    f.write(script_content)
print("📝 输出目录设置脚本已创建")

# 3. 复制styles.csv文件（如果存在）
# 这允许您从A1111迁移已保存的样式设置
styles_source = "/content/drive/MyDrive/StableDiffusion/styles.csv"
styles_dest = "/content/stable-diffusion-webui-forge/styles.csv"

if os.path.exists(styles_source):
    !cp "{styles_source}" "{styles_dest}"
    print("📋 已复制styles.csv文件")
else:
    print("ℹ️ 未找到styles.csv文件，跳过")

# 4. 执行输出目录设置脚本
print("\n🔧 正在配置输出目录...")
!python /content/stable-diffusion-webui-forge/scripts/set_output_dir.py

# 5. 启动WebUI
print("\n🚀 正在启动Stable Diffusion WebUI Forge...")
print("="*60)
print("请等待出现访问链接...")
print("="*60 + "\n")

# 启动命令及参数说明：
# --share: 创建公共链接，可以从任何地方访问
# --no-half-vae: 禁用VAE半精度，提高稳定性
# --disable-nan-check: 禁用NaN检查，提高性能
# --ckpt-dir: 指定模型文件目录
# --embeddings-dir: 指定嵌入文件目录
# --lora-dir: 指定LoRA文件目录
# --vae-dir: 指定VAE文件目录
# --controlnet-dir: 指定ControlNet模型目录

!python launch.py --share --no-half-vae --disable-nan-check \
  --ckpt-dir "/content/drive/MyDrive/StableDiffusion/checkpoints" \
  --embeddings-dir "/content/drive/MyDrive/StableDiffusion/embeddings" \
  --lora-dir "/content/drive/MyDrive/StableDiffusion/lora" \
  --vae-dir "/content/drive/MyDrive/StableDiffusion/vae" \
  --controlnet-dir "/content/drive/MyDrive/StableDiffusion/controlnet"

# 注意：您可以根据需要添加其他启动参数，例如：
# --xformers: 启用xformers内存优化（如果可用）
# --medvram: 中等VRAM优化（6-8GB显存）
# --lowvram: 低VRAM优化（4-6GB显存）
# --theme dark: 使用深色主题
# --gradio-auth username:password: 设置访问密码